# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Paillasse d'expérimentation
---

## Sujet : Prédiction BoW/RL pour tous les labels

---


## Initialisation de la paillasse 
---

In [2]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [3]:
import importlib
import pandas as pd

## Observations et environnement
---

### Chargement du corpus

In [4]:
corpus_filename = 'tmp/221003-CorpusLabel.csv'

In [5]:
df_corpus = pd.read_csv(f'data/{corpus_filename}')
df_corpus = df_corpus.convert_dtypes()

/var/folders/n0/92k02lss71s67fxfyc0qn84m0000gn/T/ipykernel_49467/3410691136.py:1: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_corpus = pd.read_csv(f'data/{corpus_filename}')


In [7]:
df_corpus[df_corpus['url'].isna()]

,url,title,article,tag,abstract,author,date,date_iso,site,text,...,trop_mots,trop_chiffres,sophisme,inversion_preuve,inverifiable,propos_raportes,sources_citees,titre_decale,denigrement,cherry_picking


In [27]:
df_corpus[df_corpus['infox'].notna()]

,url,title,article,tag,abstract,author,date,date_iso,site,text,...,trop_mots,trop_chiffres,sophisme,inversion_preuve,inverifiable,propos_raportes,sources_citees,titre_decale,denigrement,cherry_picking
3273,https://fr.novopress.info/227346/immigration-c...,Immigration clandestine : répartir ou renvoyer...,Comment faire face à l’immigration clandestine...,"['Étiquettes crise alimentaire, immigration cl...",<NA>,Novopress,1 juillet 2022,2022-07-01T00:00:00,fr.novopress.info,Immigration clandestine : répartir ou renvoyer...,...,0,0,0,0,0,0,1,0,<NA>,<NA>
3283,https://fr.novopress.info/227381/lalgerie-un-p...,"L’Algérie, un pays jeune gouverné par des viei...","L’Algérie (46 millions d’habitants en 2022), e...","['Étiquettes Algérie, Bernard Lugan']",<NA>,Novopress,6 juillet 2022,2022-07-06T00:00:00,fr.novopress.info,"L’Algérie, un pays jeune gouverné par des viei...",...,0,0,0,0,0,0,0,0,<NA>,<NA>
3289,https://fr.novopress.info/227410/un-president-...,"Un président réélu : et après ?, par Philippe ...",On ne peut pas s’empêcher de penser qu’avec ce...,"['Étiquettes Emmanuel Macron, Philippe Bilger,...",<NA>,Novopress,7 juillet 2022,2022-07-07T00:00:00,fr.novopress.info,"Un président réélu : et après ?, par Philippe ...",...,0,0,0,0,0,0,0,0,<NA>,<NA>
3641,https://fr.novopress.info/actualite-politique-...,Qui est Alexandre Douguine? Anatomie d’une pen...,L’assassinat de Daria Douguine a conduit de no...,"['Étiquettes Alexandre Douguine, Revue Elément...",<NA>,Novopress,29 août 2022,2022-08-29T00:00:00,fr.novopress.info,Qui est Alexandre Douguine? Anatomie d’une pen...,...,0,0,0,0,0,0,0,0,0,<NA>
5951,https://fr.sott.net/article/39998-Le-Pentag,Le Pentagone organise la victoire de l'Ukraine...,Le festival de l'Eurovision 2022 s'est tenu à ...,[],<NA>,<NA>,<NA>,<NA>,fr.sott.net,Le Pentagone organise la victoire de l'Ukraine...,...,0,0,0,0,1,0,0,1,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88172,https://www.wikistrike.com/2022/08/les-vaccine...,Les vaccinés infectés au Covid restent contagi...,Une étude publiée le 21 juin 2022 par le New E...,[],<NA>,<NA>,Publié par wikistrike.com ...,2022-08-20T00:00:00,www.wikistrike.com,Les vaccinés infectés au Covid restent contagi...,...,0,1,0,0,0,1,1,0,1,<NA>
88198,https://www.wikistrike.com/2022/08/washington-...,Washington enverrait des mercenaires afghans c...,Des spécialistes originaires d'Afghanistan par...,[],<NA>,<NA>,Publié par wikistrike.com ...,2022-08-20T00:00:00,www.wikistrike.com,Washington enverrait des mercenaires afghans c...,...,0,0,0,1,1,1,0,0,0,<NA>
88206,https://www.wikistrike.com/2022/09/11-septembr...,11 septembre 2001 : La théorie du « complot pi...,Le 11 septembre pour les nuls Est-il vrai que ...,[],<NA>,<NA>,Publié par wikistrike.com ...,2022-09-29T00:00:00,<NA>,<NA>,...,1,0,0,0,0,1,0,0,0,<NA>
88211,https://www.wikistrike.com/2022/09/coupures-el...,Coupures électriques imminentes en France,"Selon Reuters, le gouvernement français s'appr...",[],<NA>,<NA>,Publié par wikistrike.com ...,2022-09-29T00:00:00,<NA>,<NA>,...,0,0,0,0,0,1,0,1,0,<NA>


## Expérience
---

### Céation bags of Words

#### Mots non signifiants

In [22]:
import patat.ml.lex_analyser

lex = patat.ml.lex_analyser.LexAnalyser()

def merge_texts(texts):
    result = ''
    for text in texts:
        result = result + text + '\n'
    return result

def count_words_label(df,label):
    df = df[df[label].notna()]
    df_texts = pd.pivot_table(df, values='text', index=None, columns=label, aggfunc=merge_texts)
    wc = {}
    word_analysis = {}
    for key in df_texts.keys():
        text = df_texts[key]['text']
        count_colname = label+'_'+str(key)
        wc[key] = lex.count_tokens(lex.get_words(text))
        for word in wc[key]:
            word_dic = word_analysis.get(word,{})
            word_dic[count_colname]=wc[key][word]
            word_analysis[word]=word_dic
    return word_analysis

In [23]:
import patat.imp.labels

In [24]:
label_names = patat.imp.labels.labels

In [25]:
count_words_label(df_corpus,'infox')

{}

In [26]:
pd.pivot_table(df_corpus, values='text', index=None, columns='signe', aggfunc=merge_texts)

signe
text


In [9]:
df_count = pd.DataFrame(count_words_label(df_corpus,'infox')).T
df_count = df_count.fillna(0)

In [10]:
df_count.shape

(0, 0)

In [11]:
df_count

""


#### Identification des mots rares

In [ ]:
occ_rare = 3
def is_rare(row):
    return row['infox_0.0'] < occ_rare and row['infox_1.0'] < occ_rare

df_rare = df_count[df_count.apply(is_rare,axis=1)]

rare_words = list(df_rare.index)
len(rare_words)

### Identification des mots communs

common_size = 100
top_0 = df_count.sort_values('infox_0.0',ascending=False).head(common_size).index
top_1 = df_count.sort_values('infox_1.0',ascending=False).head(common_size).index
common_words = []
for word in top_0:
    if word in top_1:
        common_words.append(word)
len(common_words)

common_words

ignore_words = common_words + rare_words

In [ ]:
common_words

#### Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
count_vectorizer = CountVectorizer(stop_words=ignore_words)
#count_vectorizer = CountVectorizer()

In [ ]:
corpus = df_corpus['text']
X = count_vectorizer.fit_transform(corpus)
y = df_corpus['infox']

### Construction des Datasets d'entrainement et de test

In [ ]:
df_train = df_corpus[df_corpus['infox'].notna()]
X_train = X[df_train.index]
y_train = df_train['infox']

In [ ]:
X_train.shape

### Entrainement du modele

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr', max_iter=1000)
classifier = LogisticRegression(max_iter=1000)

# Fit the model
classifier.fit(X_train, y_train)


### Prédictions sur le corpus

In [ ]:
df_pred = df_corpus

In [ ]:
vectorizer = predictor['descriptor']
classifier = predictor['classifier']

In [ ]:
%%time
X = vectorizer.transform(df_pred['text'])

In [ ]:
%%time
y = classifier.predict(X)
y_proba = classifier.predict_proba(X)[:,1]




In [ ]:
df_pred.loc[:,'y'] = y
df_pred.loc[:,'y_proba'] = y_proba

In [ ]:
df_pred

### Mesure et observation des résultats

In [ ]:
import numpy as np

In [ ]:
# Import seaborn
import seaborn as sns

# Apply the default theme
sns.set_theme()

In [ ]:
sns.displot(df_pred["y_proba"], binwidth=0.01)

In [ ]:
df_recueil = df_corpus[df_corpus['infox'].notna()]
y_test = df_recueil['infox']
y_pred = df_recueil['y']

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
print('Matrice de confusion')
print(cnf_matrix)
print(f'Accuracy score {metrics.accuracy_score(y_test, y_pred)*100:.2f}%')
print(f'Recall score {metrics.recall_score(y_test, y_pred)*100:.2f}%')

In [ ]:
def size(y):
    return 'S' if y<=0.25 else 'L' if y>=0.75 else 'M'

In [ ]:
df_pred['y_bin']=df_pred['y_proba'].apply(size)

In [ ]:
df_pivot_sites = pd.pivot_table(df_pred[['site','y_bin']],index='site',columns='y_bin',aggfunc=np.count_nonzero)

In [ ]:
df_pivot_sites.fillna(0,inplace=True)

In [ ]:
df_pivot_sites['total']=df_pivot_sites['S']+df_pivot_sites['M']+df_pivot_sites['L']

In [ ]:
df_pivot_sites['S_freq']=df_pivot_sites['S']/df_pivot_sites['total']
df_pivot_sites['L_freq']=df_pivot_sites['L']/df_pivot_sites['total']

In [ ]:
df_pivot_sites

## Sauvegarde des résultats
---

In [ ]:
predict_filename = '221002-CorpusNewsLabelPred.csv'
df_pred.to_csv(f'data/corpus/{predict_filename}',index=False)

## Conclusions
---

- Production Corpus Labelisé avec prédictions : OK

## Bricolages
---

In [ ]:
df_pred